In [ ]:
import os
import sys

In [ ]:
# 把我們自己寫的模組的位置，加入到模組搜尋路徑之中，不然會有 import error
module_dir = os.path.join(os.path.dirname(os.getcwd()), 'modules')
if not module_dir in sys.path:
    sys.path.append(module_dir)

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import pickle

In [ ]:
# 載入計算回測指標的函式
from backtest import indicators

In [ ]:
# 用來讀入 all_strategies 裡面的策略
from inspect import getmembers, isfunction
import all_strategies

In [ ]:
def apply_strategy(strategy, df):
    return strategy(df)

In [ ]:
# 讀出預先下載好的股價資料
with open('../data/twstockdata', 'rb') as f:
    data = pickle.load(file=f)

# 計算各支股票的回測結果
results = []

strategies = [member[1] for member in getmembers(all_strategies) if isfunction(member[1])]

for symbol in data:
    for strategy in strategies:
        try:
            apply_strategy(strategy, data[symbol])
            if np.all(data[symbol]['signals']==0):
                print("Symbol:", symbol, "使用", strategy.__name__, "策略沒有出現買賣訊號。")
                continue
            SharpeRatio, maxdd, maxddd, finalRet = indicators(data[symbol])
            days = (data[symbol].index[-1] - data[symbol].index[0]).days
            results.append((SharpeRatio, maxdd, maxddd, finalRet, days,
                            data[symbol][data[symbol]['signals'] > 0]['signals'].sum(), symbol, strategy.__name__))
        except Exception as e:
            print("Error occurs at symbol:", symbol, "Strategy:", strategy.__name__, "==>", e.args)


results_df = pd.DataFrame(results, columns=['sharpe','MaxDrawDown','MaxDrawDownDuration','returns','days', 'entries','symbol','strategy'])

In [ ]:
# Sorted by MaxDrawDown:
results_df.sort_values('MaxDrawDown',ascending=False).head()

In [ ]:
# Sorted by returns:
results_df.sort_values('returns',ascending=False).head()

In [ ]:
# Sorted by sharpe:
results_df.sort_values('sharpe',ascending=False).head()

In [ ]:
# Sorted by MaxDrawDownDuration:
results_df.sort_values('MaxDrawDownDuration',ascending=True).head()